In [1]:
from dataloader import LeukemiaLoader
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
print(torch.cuda.is_available())
device=torch.device('cuda',0)
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms,models
import torch.nn as nn
import torch.optim as optim
import copy
from PIL import Image

True


In [2]:
class Res18_basic_Block(nn.Module):
    def __init__(self, inchannel, outchannel, stride = 1):
        super(Res18_basic_Block, self).__init__()
        
        self.basic = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(outchannel)
        )
        
        self.shortcut = nn.Sequential()
        
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride),
                nn.BatchNorm2d(outchannel)
            )
            
    def forward(self, x):
        
        out = self.basic(x)
        out = out + self.shortcut(x)
        out = nn.ReLU()(out)
        
        return out
        

In [3]:
import torch.nn as nn

class Bottleneck(nn.Module):
    
    def __init__(self, inchannel, outchannel, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.expansion = 4
        self.basic = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=1),  
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),  
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=stride, padding=1),  
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel * self.expansion, kernel_size=1, stride=1),   
            nn.BatchNorm2d(outchannel * self.expansion)
        )
         
        self.downsample = downsample
        
    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)
    
        out = self.basic(x)

        out += identity
        out = nn.ReLU()(out)

        return out


In [4]:
class ResNet_18(nn.Module):
    def __init__(self):
        super(ResNet_18, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.layer1 = self.make_layer(64, 2, stride=1)
        self.layer2 = self.make_layer(128, 2, stride=2)
        self.layer3 = self.make_layer(256, 2, stride=2)        
        self.layer4 = self.make_layer(512, 2, stride=2)  
        self.classify = nn.Linear(512, 2)
            
       
    def make_layer(self, channel, num_blocks, stride):
        
        strides = [stride] + [1]
        layers = []
        for stride in strides:
            layers.append(Res18_basic_Block(self.inchannel, channel, stride))
            self.inchannel = channel
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = nn.AdaptiveAvgPool2d((1, 1))(out)
        out = out.view(out.size(0), -1)
        out = self.classify(out)
        return out

In [5]:
class ResNet_50(nn.Module):
    def __init__(self):
        super(ResNet_50, self).__init__()
        self.inchannel = 64
        self.expansion = 4
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.num = [3,4,6,3]
        self.layer1 = self.make_layer(64, self.num[0], stride=1)
        self.layer2 = self.make_layer(128, self.num[1], stride=2)
        self.layer3 = self.make_layer(256, self.num[2], stride=2)        
        self.layer4 = self.make_layer(512, self.num[3], stride=2)  
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.classify = nn.Linear(512*4, 2)
            
       
    def make_layer(self,channel, num_blocks, stride):
        
        downsample = None
        if stride != 1 or self.inchannel != channel * self.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inchannel, channel * self.expansion, 1,stride),
                nn.BatchNorm2d(channel * self.expansion)
            )
            
        layers = []
        layers.append(Bottleneck(self.inchannel, channel, downsample=downsample, stride=stride)) 
        self.inchannel = channel*self.expansion   

        for _ in range(1, num_blocks):  
            layers.append(Bottleneck(self.inchannel,channel))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = nn.AdaptiveAvgPool2d((1, 1))(out)
        out = out.view(out.size(0), -1)
        out = self.classify(out)
        return out

In [6]:
class ResNet_152(nn.Module):
    def __init__(self):
        super(ResNet_152, self).__init__()
        self.inchannel = 64
        self.expansion = 4
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.num = [3,8,36,3]
        self.layer1 = self.make_layer(64, self.num[0], stride=1)
        self.layer2 = self.make_layer(128, self.num[1], stride=2)
        self.layer3 = self.make_layer(256, self.num[2], stride=2)        
        self.layer4 = self.make_layer(512, self.num[3], stride=2)  
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.classify = nn.Linear(512*4, 2)
            
       
    def make_layer(self,channel, num_blocks, stride):
        
        downsample = None
        if stride != 1 or self.inchannel != channel * self.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inchannel, channel * self.expansion, 1,stride),
                nn.BatchNorm2d(channel * self.expansion)
            )
            
        layers = []
        layers.append(Bottleneck(self.inchannel, channel, downsample=downsample, stride=stride)) 
        self.inchannel = channel*self.expansion   

        for _ in range(1, num_blocks):  
            layers.append(Bottleneck(self.inchannel,channel))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = nn.AdaptiveAvgPool2d((1, 1))(out)
        out = out.view(out.size(0), -1)
        out = self.classify(out)
        return out

In [7]:
def train_and_test(model, loader_train,loader_test):   
    lr = 1e-2        
    loss = torch.nn.CrossEntropyLoss()
    total_train = [0]
    total_test = [0]

    Epochs = 10     
    model=model()
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(1, Epochs+1):
        print(epoch)
        model.train()
        all_loss = 0
        all_acc = 0
        counter = 0
        print("in train")
        for idx,(data,label, path) in enumerate(loader_train):
            counter+=1
            data=data.to(device,dtype=torch.float)
            #print(data)
            #print(label)
            #print(u)
            label=label.to(device,dtype=torch.long)
            pred_y = model(data)
            mono_loss = loss(pred_y, label)
            all_loss += mono_loss
            mono_acc = evaluate(pred_y, label)
            all_acc += mono_acc
            optimizer.zero_grad()
            mono_loss.backward() 
            optimizer.step()  
        acc = all_acc/counter
        total_train.append(acc)
        print(f"Epoch: {epoch}, Loss: {all_loss/len(loader_train.dataset)}, Accuracy: {acc:.2f}%")
        print("in test")    
        #test    
        model.eval()
        all_acc = 0
        counter = 0
        for idx,(data,label, path) in enumerate(loader_test):
            counter+=1
            data=data.to(device,dtype=torch.float)
            label=label.to(device,dtype=torch.long)
            pred_y = model(data)
            mono_acc = evaluate(pred_y, label)
            all_acc+=mono_acc
        acc = all_acc/counter    
        total_test.append(acc)
        print(f"Test Accuracy: {acc:.2f}%")

        print("out test")    

    return total_train, total_test

In [8]:
def evaluate(pred_y, label):
    pred_y = pred_y.to(device)
    label = label.to(device)
    correct = pred_y.max(dim=1)[1].eq(label).sum().item()
    total = label.shape[0]
    accuracy = (correct / total) * 100
    return accuracy

def save_result(csv_path, predict_result):
    df = pd.read_csv(csv_path)
    new_df = pd.DataFrame()
    new_df['ID'] = df['Path']
    new_df["label"] = predict_result
    new_df.to_csv("./your_student_id_resnet18.csv", index=False)
def plot(res18_train,res18_test, res50_train,res50_test, res152_train,res152_test):
    epoch_list = []
    for i in range(len(res18_train)):
        epoch_list.append(i)

    plt.plot(epoch_list, res18_train, label= "res18_train")
    plt.plot(epoch_list, res18_test, label= "res18_test")
    plt.plot(epoch_list, res50_train, label= "res50_train")
    plt.plot(epoch_list, res50_test, label= "res50_test")
    plt.plot(epoch_list, res152_train, label = "res152_train")
    plt.plot(epoch_list, res152_test, label = "res152_test")
    plt.ylim(50, 100)
    plt.legend(loc = 4)
    plt.show()

In [9]:
from torch.utils.data import DataLoader, SubsetRandomSampler
if __name__ == "__main__":
    batch_size = 4
    sampler = SubsetRandomSampler(range(1000))
    dataset_train=LeukemiaLoader(root='new_dataset/train/',mode='train')
    loader_train=DataLoader(dataset=dataset_train,sampler = sampler, batch_size=batch_size,shuffle=False,num_workers=4)
    #img, label, path = dataset_train.__getitem__(0)
    #print(path)
    #print(img[0])
    dataset_valid=LeukemiaLoader(root='new_dataset/valid/',mode='valid')
    loader_valid=DataLoader(dataset=dataset_valid,sampler = sampler, batch_size=batch_size,shuffle=False,num_workers=4)

    dataset_test=LeukemiaLoader(root='new_dataset/test/',mode='test')
    loader_test=DataLoader(dataset=dataset_test,batch_size=batch_size,shuffle=False,num_workers=4)
    
    neural_network_18 = ResNet_18()
    res18_train,res18_test= train_and_test(ResNet_18, loader_train,loader_valid)

    neural_network_50 = ResNet_50()
    res50_train,res50_test= train_and_test(ResNet_50, loader_train,loader_valid)

    neural_network_152 = ResNet_152()
    res152_train,res152_test= train_and_test(ResNet_152, loader_train,loader_valid)


> Found 7995 images...
> Found 1599 images...
> Found 1067 images...


1
in train
Epoch: 1, Loss: 0.13715076446533203, Accuracy: 74.04%
in test
Test Accuracy: 72.67%
out test
2
in train
Epoch: 2, Loss: 0.12269831448793411, Accuracy: 77.10%
in test
Test Accuracy: 72.48%
out test
3
in train
Epoch: 3, Loss: 0.11509065330028534, Accuracy: 79.42%
in test
Test Accuracy: 77.17%
out test
4
in train
Epoch: 4, Loss: 0.10999676585197449, Accuracy: 80.65%
in test
Test Accuracy: 82.04%
out test
5
in train
Epoch: 5, Loss: 0.10221098363399506, Accuracy: 82.54%
in test
Test Accuracy: 71.67%
out test
6
in train
Epoch: 6, Loss: 0.09688196331262589, Accuracy: 83.58%
in test
Test Accuracy: 76.17%
out test
7
in train
Epoch: 7, Loss: 0.0921986922621727, Accuracy: 84.13%
in test
Test Accuracy: 83.23%
out test
8
in train
Epoch: 8, Loss: 0.08850770443677902, Accuracy: 84.89%
in test
Test Accuracy: 68.23%
out test
9
in train
Epoch: 9, Loss: 0.08478092402219772, Accuracy: 85.88%
in test
Test Accuracy: 83.62%
out test
10
in train
Epoch: 10, Loss: 0.08044611662626266, Accuracy: 86.63

KeyboardInterrupt: 